In [1]:
import acquire
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import re
from time import strftime
from time import sleep
import os
import json
import unicodedata
from typing import Dict, List, Optional, Union, cast
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from requests import get

from env import github_token, github_username

In [2]:
url = 'https://github.com/hoppscotch/hoppscotch'
def stuff(url):
    response = get(url, headers = {'User-Agent': 'Codeup Data Science'})
    soup = BeautifulSoup(response.text, features='lxml')
    speech = soup.select('.markdown-body')
    speech = speech[0].select('p')
    
    words = [words.text for words in speech]
    
    return words

In [21]:
words = stuff(url)

In [22]:
words

['\n',
 '',
 '\n\n      Open source API development ecosystem\n    \n',
 '\n',
 '   ',
 '',
 '\n\n      Built with ❤︎ by\n      \n        contributors\n      \n\n',
 '\n\n\n\n',
 ' ',
 '❤️ Lightweight: Crafted with minimalistic UI design.',
 '⚡️ Fast: Send requests and get/copy responses in real-time.',
 'HTTP Methods',
 '🌈 Make it yours: Customizable combinations for background, foreground and accent colors — customize now ✨.',
 'Theming',
 'Customized themes are synced with cloud / local session',
 '🔥 PWA: Install as a PWA on your device.',
 '🚀 Request: Retrieve response from endpoint instantly.',
 '🔌 WebSocket: Establish full-duplex communication channels over a single TCP connection.',
 '📡 Server Sent Events: Receive a stream of updates from a server over a HTTP connection without resorting to polling.',
 '🌩 Socket.IO: Send and Receive data with SocketIO server.',
 '🦟 MQTT: Subscribe and Publish to topics of a MQTT Broker.',
 '🔮 GraphQL: GraphQL is a query language for APIs and a r

In [3]:
def clean_speech(words):
    original = ' '.join(words)
    text = original.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r"[^a-z0-9'\s]", '', text)
    wnl = nltk.stem.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in text.split()]
    text_lemma = ' '.join(lemmas)
    stopwords = nltk.corpus.stopwords.words('english')
    newStopWords = ['u','ha','wa']
    stopwords.extend(newStopWords)
    words = text_lemma.split()
    filtered_words = [w for w in words if w not in stopwords]
    speech = ' '.join(filtered_words)

    return speech

In [23]:
speech = clean_speech(words)
speech

'open source api development ecosystem built contributor lightweight crafted minimalistic ui design fast send request getcopy response realtime http method make customizable combination background foreground accent color customize theming customized theme synced cloud local session pwa install pwa device request retrieve response endpoint instantly websocket establish fullduplex communication channel single tcp connection server sent event receive stream update server http connection without resorting polling socketio send receive data socketio server mqtt subscribe publish topic mqtt broker graphql graphql query language apis runtime fulfilling query existing data authorization allows identify end user header describes format body request sent parameter use request parameter set varying part simulated request request body used send receive data via rest api response contains status line header messageresponse body history request entry synced cloud local session storage restore single

In [ ]:
df.readme_contents

In [4]:
df = pd.read_json('data.json')
df.head()

,repo,language,readme_contents
0,thedaviddias/Front-End-Checklist,None,"<h1 align=""center"">\n<br>\n <img src=""https:/..."
1,onevcat/Kingfisher,Swift,"<p align=""center"">\n<img src=""https://raw.gith..."
2,FallibleInc/security-guide-for-developers,None,# 实用性 WEB 开发人员安全须知 \n\n### 目标读者 \n\n安全问题主要由以...
3,tailwindlabs/tailwindcss,JavaScript,"<p>\n <a href=""https://tailwindcss.com/"" ta..."
4,codepath/android_guides,None,# CodePath Android Cliffnotes\n\nWelcome to th...


In [5]:
df.language.value_counts(dropna = False)

JavaScript          141
Python               59
NaN                  54
Java                 46
TypeScript           45
Go                   36
C++                  30
C                    20
Swift                13
Shell                13
HTML                 12
Rust                 11
Ruby                  9
Kotlin                9
C#                    9
Vim script            6
Vue                   6
PHP                   6
Jupyter Notebook      6
CSS                   5
Objective-C           3
TeX                   2
CoffeeScript          2
Elixir                2
Haskell               2
Lua                   2
Batchfile             1
OCaml                 1
Dart                  1
Emacs Lisp            1
Standard ML           1
Crystal               1
Clojure               1
Rascal                1
AsciiDoc              1
Dockerfile            1
Makefile              1
Name: language, dtype: int64

In [6]:
#sums null counts
df.isnull().sum()

repo                0
language           54
readme_contents     0
dtype: int64

In [7]:
df.shape

(560, 3)

In [8]:
#drops nulls
df = df.dropna()
df.shape

(506, 3)

In [9]:
def is_chinese(texts):
    '''
    This function takes in a dataframe and return true if the scanned text is in chinese
    '''
    if re.search("[\u4e00-\u9FFF]", texts):
            return True

In [10]:
#drops true values from is_chinese function
df = df[df.readme_contents.apply(is_chinese) !=True]

In [11]:
df.shape

(394, 3)

In [12]:
df.head(5)

,repo,language,readme_contents
3,tailwindlabs/tailwindcss,JavaScript,"<p>\n <a href=""https://tailwindcss.com/"" ta..."
5,github/fetch,JavaScript,# window.fetch polyfill\n\nThe `fetch()` funct...
8,Kong/insomnia,JavaScript,# Insomnia REST Client\n\n[![Slack Channel](ht...
10,ggreer/the_silver_searcher,C,# The Silver Searcher\n\nA code searching tool...
11,ReactiveX/RxSwift,Swift,"<p align=""center"">\n<img src=""assets/RxSwift_L..."


In [13]:
#reset index 
df = df.reset_index(drop = True)
df.head()

,repo,language,readme_contents
0,tailwindlabs/tailwindcss,JavaScript,"<p>\n <a href=""https://tailwindcss.com/"" ta..."
1,github/fetch,JavaScript,# window.fetch polyfill\n\nThe `fetch()` funct...
2,Kong/insomnia,JavaScript,# Insomnia REST Client\n\n[![Slack Channel](ht...
3,ggreer/the_silver_searcher,C,# The Silver Searcher\n\nA code searching tool...
4,ReactiveX/RxSwift,Swift,"<p align=""center"">\n<img src=""assets/RxSwift_L..."


In [24]:
def basic_clean(string):
    '''
    This function takes in a string and normalizes it for nlp purposes
    '''
    # lowercase the string
    string = string.lower()

    # return normal form for the unicode string, encode/remove ascii
    string = unicodedata.normalize('NFKD', string).encode('ascii', 'ignore').decode('utf-8')
    
    # breaks down the string by keeping alphabet letters, numbers, apostraphes and spaces
    string = re.sub(r"[^a-z0-9\s]", '', string)
    
    return string


def tokenize(string):
    '''
    This function takes in a string and tokenizes it
    '''
    # create the tokenizer
    tokenizer = nltk.tokenize.ToktokTokenizer()
    
    # use the tokenizer, return as a string
    string = tokenizer.tokenize(string, return_str = True)
    
    return string

def stem(text):
    '''
    This function takes in a text and stems the words to their original stem
    '''
    
    # create a porter stemmer
    ps = nltk.porter.PorterStemmer()
    
    # loop through the text to stem the words
    stems = [ps.stem(word) for word in text.split()]
    
    # return back together
    stems = ' '.join(stems)
    
    return stems


def lemmatize(text):
    '''
    This function takes in a text and changes the words back to their root (lemmatize)
    '''
    
    # create the lemmatizer
    wnl = nltk.stem.WordNetLemmatizer()
    
    # loop through the list to split and lemmatize
    lemmas = [wnl.lemmatize(word) for word in text.split()]
    
    # return back together
    lemmas =' '.join(lemmas)
    
    return lemmas


def remove_stopwords(string, extra_words = [], exclude_words = []):
    '''
    This function takes in a string
    And returns the string with the English stopwords removed
    Additional stopwords can be added to extra_words (list)
    or words to exclude from stopwords can be added to exclude_words (list)
    
    -- This might break if the excluded words aren't in the stopwords list
    '''
    # define stopwords list      
    stopwords_list = stopwords.words('English')
    
    # add or remove words based on arguments
    stopwords_list = set(stopwords_list) - set(exclude_words) # the set removes words
    
    stopwords_list = stopwords_list.union(set(extra_words))
        
    # remove stopwords from string
    # turn string into list
    words = string.split()
    
    # remove the stopwords
    filtered_words = [w for w in words if w not in stopwords_list]
    
    # turn back into a string
    new_string = ' '.join(filtered_words)
    
    return new_string


################## ~~~~~~ Mother Prep Function ~~~~~~ ##################

def prepare_nlp_data(df, content = 'content', extra_words=[], exclude_words=[]):
    '''
    This function take in a df and the content (in string) for the column 
    with an option to pass lists for additional stopwords (extra_words)
    and an option to pass words to exclude from stopwords (exclude words)
    returns a df with the  original text, cleaned (tokenized and stopwords removed),
    stemmed text, lemmatized text.
    '''
    df['clean'] = df[content].apply(basic_clean)\
                            .apply(tokenize)\
                            .apply(remove_stopwords, 
                                   extra_words=extra_words, exclude_words=exclude_words)
    
    df['stemmed'] = df['clean'].apply(stem)
    
    df['lemmatized'] = df['clean'].apply(lemmatize)
    
    return df


def is_chinese(texts):
    '''
    This function takes in a dataframe and return true if the scanned text is in chinese
    '''
    if re.search("[\u4e00-\u9FFF]", texts):
            return True



def get_top_4_languages(df):
    '''
    This function takes in a dataframe and returns the top four
    programming languages found in the data
    '''
    top_4_list = list(df.language.value_counts().head(4).index)
    mask = df.language.apply(lambda x: x in top_4_list)
    df = df[mask]
    return df


def drop_unneeded_data(df):
    '''
    This function takes in the repo dataframe
    Drops any rows with nulls
    Drops any rows that are chinese
    Drops all rows that aren't in the top 4 languages
    '''
    df = df.dropna()
    df = df[df.readme_contents.apply(is_chinese) !=True]
    df = get_top_4_languages(df)
    df = df.reset_index().drop(columns = 'index')
    return df


def split_data(df):
    '''
    This function takes in a dataframe and splits it into train, test, and 
    validate dataframes for my model
    '''

    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, stratify=df.language)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, stratify=train_validate.language)

    print('train--->', train.shape)
    print('validate--->', validate.shape)
    print('test--->', test.shape)
    return train, validate, test

In [26]:
df = prepare_nlp_data(df, content = 'readme_contents', extra_words=['customer', 'customers', '1', '2'])

In [27]:
df.head()

,repo,language,readme_contents,clean,stemmed,lemmatized
0,tailwindlabs/tailwindcss,JavaScript,"<p>\n <a href=""https://tailwindcss.com/"" ta...",p hrefhttpstailwindcsscom targetblank img altt...,p hrefhttpstailwindcsscom targetblank img altt...,p hrefhttpstailwindcsscom targetblank img altt...
1,github/fetch,JavaScript,# window.fetch polyfill\n\nThe `fetch()` funct...,windowfetch polyfill fetch function promisebas...,windowfetch polyfil fetch function promisebas ...,windowfetch polyfill fetch function promisebas...
2,Kong/insomnia,JavaScript,# Insomnia REST Client\n\n[![Slack Channel](ht...,insomnia rest client slack channelhttpschatins...,insomnia rest client slack channelhttpschatins...,insomnia rest client slack channelhttpschatins...
3,ggreer/the_silver_searcher,C,# The Silver Searcher\n\nA code searching tool...,silver searcher code searching tool similar ac...,silver searcher code search tool similar ack f...,silver searcher code searching tool similar ac...
4,ReactiveX/RxSwift,Swift,"<p align=""center"">\n<img src=""assets/RxSwift_L...",p aligncenter img srcassetsrxswiftlogopng widt...,p aligncent img srcassetsrxswiftlogopng width3...,p aligncenter img srcassetsrxswiftlogopng widt...


#### prep function

In [ ]:
def prep():
#drop nulls
    df = df.dropna()
#drop written chinese
    df = df[df.readme_contents.apply(is_chinese) !=True]
#Keeps top 5 languages
    df = top_5
#reindex and drop old index column
    df = df.reset_index(drop = True)
    
    return df

In [14]:
text = df['readme_contents'].iloc[0]

In [15]:
text

'<p>\n    <a href="https://tailwindcss.com/" target="_blank">\n      <img alt="Tailwind CSS" width="350" src="https://refactoringui.nyc3.cdn.digitaloceanspaces.com/tailwind-logo-sticker.svg">\n    </a><br>\n    A utility-first CSS framework for rapidly building custom user interfaces.\n</p>\n\n<p>\n    <a href="https://github.com/tailwindlabs/tailwindcss/actions"><img src="https://img.shields.io/github/workflow/status/tailwindlabs/tailwindcss/Node.js%20CI" alt="Build Status"></a>\n    <a href="https://www.npmjs.com/package/tailwindcss"><img src="https://img.shields.io/npm/dt/tailwindcss.svg" alt="Total Downloads"></a>\n    <a href="https://github.com/tailwindcss/tailwindcss/releases"><img src="https://img.shields.io/npm/v/tailwindcss.svg" alt="Latest Release"></a>\n    <a href="https://github.com/tailwindcss/tailwindcss/blob/master/LICENSE"><img src="https://img.shields.io/npm/l/tailwindcss.svg" alt="License"></a>\n</p>\n\n------\n\n## Documentation\n\nFor full documentation, visit [ta

In [17]:
text = text.lower()
text

'<p>\n    <a href="https://tailwindcss.com/" target="_blank">\n      <img alt="tailwind css" width="350" src="https://refactoringui.nyc3.cdn.digitaloceanspaces.com/tailwind-logo-sticker.svg">\n    </a><br>\n    a utility-first css framework for rapidly building custom user interfaces.\n</p>\n\n<p>\n    <a href="https://github.com/tailwindlabs/tailwindcss/actions"><img src="https://img.shields.io/github/workflow/status/tailwindlabs/tailwindcss/node.js%20ci" alt="build status"></a>\n    <a href="https://www.npmjs.com/package/tailwindcss"><img src="https://img.shields.io/npm/dt/tailwindcss.svg" alt="total downloads"></a>\n    <a href="https://github.com/tailwindcss/tailwindcss/releases"><img src="https://img.shields.io/npm/v/tailwindcss.svg" alt="latest release"></a>\n    <a href="https://github.com/tailwindcss/tailwindcss/blob/master/license"><img src="https://img.shields.io/npm/l/tailwindcss.svg" alt="license"></a>\n</p>\n\n------\n\n## documentation\n\nfor full documentation, visit [ta

In [20]:
soup = BeautifulSoup(response.text, features='lxml')
speech = soup.select('.markdown-body')
text = text[0].select('p')

AttributeError: 'str' object has no attribute 'text'

In [ ]:
text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
text = re.sub(r"[^a-z0-9'\s]", '', text)
wnl = nltk.stem.WordNetLemmatizer()
lemmas = [wnl.lemmatize(word) for word in text.split()]
text_lemma = ' '.join(lemmas)
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['u','ha','wa']
stopwords.extend(newStopWords)
words = text_lemma.split()
filtered_words = [w for w in words if w not in stopwords]
speech = ' '.join(filtered_words)

In [ ]:
speech